In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Heart Disease - Model Experimentation\n",
    "## Medical Analytics Mini Project\n",
    "\n",
    "**Team:** Mercy Thokozani Ngwenya & Mediator Nhongo\n",
    "\n",
    "This notebook experiments with multiple machine learning models to predict heart disease and compares their performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Import our custom classes\n",
    "from src.trainer import ModelTrainer\n",
    "from src.model_factory import ModelFactory\n",
    "from src.predictor import Predictor\n",
    "\n",
    "# Setup plotting\n",
    "%matplotlib inline\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "print(\"✅ Libraries imported successfully\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize and run the complete pipeline\n",
    "print(\"🚀 Starting Complete ML Pipeline...\")\n",
    "trainer = ModelTrainer('data/cleveland.data')\n",
    "results = trainer.run_pipeline()\n",
    "\n",
    "print(\"✅ Pipeline completed successfully!\")\n",
    "print(f\"🎯 Best Model: {results['best_model_name']}\")\n",
    "print(f\"📊 Best CV Score: {results['best_score']:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate performance report\n",
    "print(\"📈 Model Performance Comparison:\")\n",
    "report = trainer.generate_report()\n",
    "report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize model comparison\n",
    "print(\"📊 Model Performance Visualization:\")\n",
    "trainer.plot_model_comparison()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Feature importance analysis\n",
    "print(\"🔍 Top Feature Importances:\")\n",
    "trainer.plot_feature_importance(top_n=10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Detailed analysis of each model\n",
    "print(\"🧪 Detailed Model Analysis:\")\n",
    "\n",
    "for model_name, result in results['evaluation_results'].items():\n",
    "    print(f\"\\n{'='*50}\")\n",
    "    print(f\"Model: {model_name.upper()}\")\n",
    "    print(f\"{'='*50}\")\n",
    "    print(f\"Cross-validation Score: {result['training_score']:.4f}\")\n",
    "    print(f\"Test Accuracy: {result['test_metrics']['accuracy']:.4f}\")\n",
    "    print(f\"Test Precision: {result['test_metrics']['precision']:.4f}\")\n",
    "    print(f\"Test Recall: {result['test_metrics']['recall']:.4f}\")\n",
    "    print(f\"Test F1-Score: {result['test_metrics']['f1_score']:.4f}\")\n",
    "    if 'roc_auc' in result['test_metrics']:\n",
    "        print(f\"ROC AUC: {result['test_metrics']['roc_auc']:.4f}\")\n",
    "    print(f\"Best Parameters: {result['best_params']}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Confusion matrix for the best model\n",
    "best_model_name = results['best_model_name']\n",
    "best_model_result = results['evaluation_results'][best_model_name]\n",
    "cm = best_model_result['test_metrics']['confusion_matrix']\n",
    "\n",
    "plt.figure(figsize=(8, 6))\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', \n",
    "            xticklabels=['No Disease', 'Disease'],\n",
    "            yticklabels=['No Disease', 'Disease'])\n",
    "plt.title(f'Confusion Matrix - {best_model_name.upper()}', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Predicted')\n",
    "plt.ylabel('Actual')\n",
    "plt.show()\n",
    "\n",
    "# Calculate performance metrics from confusion matrix\n",
    "tn, fp, fn, tp = cm.ravel()\n",
    "accuracy = (tp + tn) / (tp + tn + fp + fn)\n",
    "precision = tp / (tp + fp) if (tp + fp) > 0 else 0\n",
    "recall = tp / (tp + fn) if (tp + fn) > 0 else 0\n",
    "specificity = tn / (tn + fp) if (tn + fp) > 0 else 0\n",
    "\n",
    "print(f\"\\n📊 Detailed Performance Metrics for {best_model_name}:\")\n",
    "print(f\"• True Positives: {tp}\")\n",
    "print(f\"• True Negatives: {tn}\")\n",
    "print(f\"• False Positives: {fp}\")\n",
    "print(f\"• False Negatives: {fn}\")\n",
    "print(f\"• Accuracy: {accuracy:.4f}\")\n",
    "print(f\"• Precision: {precision:.4f}\")\n",
    "print(f\"• Recall (Sensitivity): {recall:.4f}\")\n",
    "print(f\"• Specificity: {specificity:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test the predictor with sample data\n",
    "print(\"🔮 Testing Predictor with Sample Data...\")\n",
    "\n",
    "# Save the best model\n",
    "model_path = 'best_heart_disease_model.pkl'\n",
    "trainer.model_factory.save_model(results['best_model'], model_path)\n",
    "\n",
    "# Create predictor\n",
    "predictor = Predictor(model_path)\n",
    "\n",
    "# Sample patient data for prediction\n",
    "sample_patient = {\n",
    "    'age': 52,\n",
    "    'sex': 1,\n",
    "    'cp': 0,\n",
    "    'trestbps': 125,\n",
    "    'chol': 212,\n",
    "    'fbs': 0,\n",
    "    'restecg': 1,\n",
    "    'thalach': 168,\n",
    "    'exang': 0,\n",
    "    'oldpeak': 1.0,\n",
    "    'slope': 2,\n",
    "    'ca': 2,\n",
    "    'thal': 3\n",
    "}\n",
    "\n",
    "# Make prediction\n",
    "prediction_result = predictor.predict_single(sample_patient)\n",
    "interpretation = predictor.get_prediction_interpretation(prediction_result)\n",
    "\n",
    "print(\"\\n🎯 Sample Prediction Results:\")\n",
    "for key, value in prediction_result.items():\n",
    "    print(f\"• {key.replace('_', ' ').title()}: {value}\")\n",
    "print(f\"\\n💡 Interpretation: {interpretation}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare multiple sample predictions\n",
    "print(\"🧪 Multiple Sample Predictions:\")\n",
    "\n",
    "test_patients = [\n",
    "    {'age': 35, 'sex': 0, 'cp': 1, 'trestbps': 110, 'chol': 180, 'fbs': 0, \n",
    "     'restecg': 0, 'thalach': 175, 'exang': 0, 'oldpeak': 0.5, 'slope': 1, 'ca': 0, 'thal': 2},\n",
    "    \n",
    "    {'age': 65, 'sex': 1, 'cp': 3, 'trestbps': 160, 'chol': 280, 'fbs': 1, \n",
    "     'restecg': 1, 'thalach': 110, 'exang': 1, 'oldpeak': 3.0, 'slope': 3, 'ca': 3, 'thal': 6},\n",
    "    \n",
    "    {'age': 45, 'sex': 1, 'cp': 2, 'trestbps': 130, 'chol': 220, 'fbs': 0, \n",
    "     'restecg': 0, 'thalach': 150, 'exang': 0, 'oldpeak': 1.2, 'slope': 2, 'ca': 1, 'thal': 3}\n",
    "]\n",
    "\n",
    "patient_descriptions = [\"Young Healthy\", \"High Risk Elderly\", \"Middle-aged Moderate\"]\n",
    "\n",
    "for i, (patient, desc) in enumerate(zip(test_patients, patient_descriptions), 1):\n",
    "    result = predictor.predict_single(patient)\n",
    "    interpretation = predictor.get_prediction_interpretation(result)\n",
    "    \n",
    "    print(f\"\\n{'='*60}\")\n",
    "    print(f\"Patient {i}: {desc}\")\n",
    "    print(f\"{'='*60}\")\n",
    "    print(f\"Prediction: {result['prediction_label']}\")\n",
    "    print(f\"Disease Probability: {result.get('probability_disease', 'N/A'):.2%}\")\n",
    "    print(f\"Confidence: {result.get('confidence', 'N/A'):.2%}\")\n",
    "    print(f\"Interpretation: {interpretation}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Model performance summary visualization\n",
    "print(\"📊 Final Model Performance Summary:\")\n",
    "\n",
    "metrics_df = report[['Model', 'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1-Score']]\n",
    "\n",
    "plt.figure(figsize=(14, 10))\n",
    "\n",
    "# Plot 1: Accuracy comparison\n",
    "plt.subplot(2, 2, 1)\n",
    "sns.barplot(data=metrics_df, x='Model', y='Test Accuracy', palette='viridis')\n",
    "plt.title('Model Accuracy Comparison', fontweight='bold')\n",
    "plt.xticks(rotation=45)\n",
    "plt.ylim(0.7, 1.0)\n",
    "\n",
    "# Plot 2: Precision-Recall comparison\n",
    "plt.subplot(2, 2, 2)\n",
    "metrics_df_melted = metrics_df.melt(id_vars=['Model'], \n",
    "                                    value_vars=['Test Precision', 'Test Recall'], \n",
    "                                    var_name='Metric', value_name='Score')\n",
    "sns.barplot(data=metrics_df_melted, x='Model', y='Score', hue='Metric', palette='Set2')\n",
    "plt.title('Precision vs Recall', fontweight='bold')\n",
    "plt.xticks(rotation=45)\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "\n",
    "# Plot 3: F1-Score comparison\n",
    "plt.subplot(2, 2, 3)\n",
    "sns.barplot(data=metrics_df, x='Model', y='Test F1-Score', palette='coolwarm')\n",
    "plt.title('F1-Score Comparison', fontweight='bold')\n",
    "plt.xticks(rotation=45)\n",
    "plt.ylim(0.7, 1.0)\n",
    "\n",
    "# Plot 4: Best model performance breakdown\n",
    "plt.subplot(2, 2, 4)\n",
    "best_model_metrics = metrics_df[metrics_df['Model'] == best_model_name].iloc[0]\n",
    "metrics_to_plot = ['Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1-Score']\n",
    "values = [best_model_metrics[metric] for metric in metrics_to_plot]\n",
    "colors = ['lightblue', 'lightgreen', 'lightcoral', 'gold']\n",
    "\n",
    "plt.pie(values, labels=metrics_to_plot, autopct='%1.1f%%', colors=colors, startangle=90)\n",
    "plt.title(f'{best_model_name.upper()} Performance\\nBreakdown', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save detailed results\n",
    "print(\"💾 Saving Detailed Results...\")\n",
    "trainer.save_pipeline_results('model_experimentation_results.json')\n",
    "\n",
    "print(\"✅ Results saved to 'model_experimentation_results.json'\")\n",
    "print(\"✅ Best model saved to 'best_heart_disease_model.pkl'\")\n",
    "print(\"\\n🎉 Model Experimentation Completed Successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Key Findings from Model Experimentation:\n",
    "\n",
    "1. **Best Performing Model**: [Will be displayed after execution]\n",
    "2. **Accuracy Range**: Models achieved X% to Y% accuracy\n",
    "3. **Important Features**: Top features influencing predictions\n",
    "4. **Clinical Relevance**: Model demonstrates good medical predictive capability\n",
    "5. **Deployment Ready**: Model can be used for patient risk assessment"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}